In [44]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_score
from sklearn.metrics import classification_report
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix,precision_score
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
forest_data = pd.read_csv('forestfires.csv')
forest_data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [3]:
forest_data.month.replace(('jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'),(1,2,3,4,5,6,7,8,9,10,11,12),inplace=True)
forest_data.day.replace(('mon','tue','wed','thu','fri','sat','sun'),(1,2,3,4,5,6,7),inplace=True)

In [4]:
forest_data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,8,7,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,8,7,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,8,7,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,8,6,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [5]:
data = forest_data[['month','day','FFMC','DMC','DC','ISI','temp','RH','wind','rain','area']]
data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...
512,8,7,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,8,7,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,8,7,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,8,6,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [6]:
data['area'].values[data['area'].values>0]=1
data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
512,8,7,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,1.0
513,8,7,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,1.0
514,8,7,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,1.0
515,8,6,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.0


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_features = data.drop(['month','day'],axis=1)
scaled_features = scaler.fit_transform(numerical_features)
scaled_features

array([[-0.80595947, -1.32332557, -1.83047676, ...,  1.49861442,
        -0.07326831, -1.04552255],
       [-0.00810203, -1.17954077,  0.48889092, ..., -1.74175564,
        -0.07326831, -1.04552255],
       [-0.00810203, -1.04982188,  0.56071545, ..., -1.51828184,
        -0.07326831, -1.04552255],
       ...,
       [-1.64008316, -0.84664771,  0.47476811, ...,  1.49861442,
        -0.07326831,  0.95645952],
       [ 0.68095666,  0.54900254,  0.26938221, ..., -0.00983371,
        -0.07326831, -1.04552255],
       [-2.02087875, -1.68591332, -1.78044169, ...,  0.26950853,
        -0.07326831, -1.04552255]])

In [25]:
x = scaled_features
y = data.iloc[:,10]

In [26]:
x

array([[-0.80595947, -1.32332557, -1.83047676, ...,  1.49861442,
        -0.07326831, -1.04552255],
       [-0.00810203, -1.17954077,  0.48889092, ..., -1.74175564,
        -0.07326831, -1.04552255],
       [-0.00810203, -1.04982188,  0.56071545, ..., -1.51828184,
        -0.07326831, -1.04552255],
       ...,
       [-1.64008316, -0.84664771,  0.47476811, ...,  1.49861442,
        -0.07326831,  0.95645952],
       [ 0.68095666,  0.54900254,  0.26938221, ..., -0.00983371,
        -0.07326831, -1.04552255],
       [-2.02087875, -1.68591332, -1.78044169, ...,  0.26950853,
        -0.07326831, -1.04552255]])

In [27]:
y

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
512    1.0
513    1.0
514    1.0
515    0.0
516    0.0
Name: area, Length: 517, dtype: float64

In [28]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.20,random_state = 12,stratify = y)

In [29]:
ridge = Ridge()
cv_score = cross_val_score( estimator=ridge,X=x,y=y,cv=5)
cv_score

array([0.        , 0.99999269, 0.99999095, 0.99999358, 0.99998665])

In [30]:
lasso = Lasso()
cv_score = cross_val_score( estimator=lasso,X=x,y=y,cv=5)
cv_score

array([ 0.        , -0.16201852, -0.33403909, -0.06511687, -0.0316329 ])

In [31]:
elasticnet = ElasticNet()
cv_score = cross_val_score( estimator=elasticnet,X=x,y=y,cv=5)
cv_score

array([ 0.        , -0.16201852, -0.33403909, -0.06374349, -0.03039895])

In [32]:
svc_classifier = SVC(kernel='linear')
svc_classifier.fit(x_train,y_train)
y_pred = svc_classifier.predict(x_test)


In [33]:
print("Overall Accuracy : ",round(accuracy_score(y_test,y_pred),4))
print("Precision        : ",round(precision_score(y_test,y_pred),4))
print("Recall           : ",round(recall_score(y_test,y_pred),4))
print("Confusion Matrix : ",(confusion_matrix(y_test,y_pred),4))

Overall Accuracy :  1.0
Precision        :  1.0
Recall           :  1.0
Confusion Matrix :  (array([[50,  0],
       [ 0, 54]], dtype=int64), 4)


In [34]:
model = GridSearchCV(estimator=svc_classifier,param_grid={ 'C':[10,15,13,12,11,0.001],'kernel':['rbf'],'gamma':[50,5,10,0.5]},cv=10)
model.fit(x_train,y_train)
print(model.best_params_)
print(model.best_score_)

{'C': 10, 'gamma': 0.5, 'kernel': 'rbf'}
0.9951800232288036


In [35]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
rf_model = RandomForestClassifier()
grid_model = GridSearchCV(estimator = rf_model,param_grid = {'n_estimators':[20,30,40,50,60],
                                                             'criterion':['gini','entropy'],
                                                             'max_depth':[2,3,4,5,6,7],
                                                             'random_state':[2,3,4,5]})
grid_model.fit(x_train,y_train)
print(grid_model.best_params_)
print(grid_model.best_score_)

{'criterion': 'gini', 'max_depth': 2, 'n_estimators': 20, 'random_state': 2}
1.0


In [37]:
rf_model = RandomForestRegressor()
grid_model = GridSearchCV(estimator = rf_model,param_grid = {'n_estimators':[20,30,40,50,60],
                                                             'criterion':['mse'],
                                                             'max_depth':[2,3,4,5,6,7],
                                                             'random_state':[2,3,4,5]})
grid_model.fit(x_train,y_train)
print(grid_model.best_params_)
print(grid_model.best_score_)

{'criterion': 'mse', 'max_depth': 2, 'n_estimators': 20, 'random_state': 2}
1.0


In [38]:
gb_model = GradientBoostingClassifier()
grid_model = GridSearchCV(estimator = gb_model,param_grid = {'n_estimators':[20,30,40,50,60],
                                                             'criterion':['friedman_mse'],
                                                             'max_depth':[2,3,4,5,6,7],
                                                             'random_state':[2,3,4,5]})
grid_model.fit(x_train,y_train)
print(grid_model.best_params_)
print(grid_model.best_score_)

{'criterion': 'friedman_mse', 'max_depth': 2, 'n_estimators': 20, 'random_state': 2}
1.0


In [39]:
gb_model.fit(x_train,y_train)
y_pred_gb = gb_model.predict(x_test)
y_pred_gb

array([0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1.,
       0., 1.])

In [40]:
print("Overall Accuracy : ",round(accuracy_score(y_test,y_pred_gb),4))
print("Precision        : ",round(precision_score(y_test,y_pred_gb),4))
print("Recall           : ",round(recall_score(y_test,y_pred_gb),4))
print("Confusion Matrix : ",(confusion_matrix(y_test,y_pred_gb),4))

Overall Accuracy :  1.0
Precision        :  1.0
Recall           :  1.0
Confusion Matrix :  (array([[50,  0],
       [ 0, 54]], dtype=int64), 4)


In [42]:
rf_model.fit(x_train,y_train)
y_pred_rf = rf_model.predict(x_test)
y_pred_rf

array([0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1.,
       0., 1.])

In [43]:
print("Overall Accuracy : ",round(accuracy_score(y_test,y_pred_rf),4))
print("Precision        : ",round(precision_score(y_test,y_pred_rf),4))
print("Recall           : ",round(recall_score(y_test,y_pred_rf),4))
print("Confusion Matrix : ",(confusion_matrix(y_test,y_pred_rf),4))

Overall Accuracy :  1.0
Precision        :  1.0
Recall           :  1.0
Confusion Matrix :  (array([[50,  0],
       [ 0, 54]], dtype=int64), 4)
